<a href="https://www.kaggle.com/simarpreetkaur652/101917177-evaluation-ds?scriptVersionId=90285795" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/prediction-stock/IXIC.csv


In [2]:
import cudf as cd
import cuml as ml
import cupy as cp
import numpy as np
import pandas as pd
import sklearn

In [3]:
# Reading the csv file and converting into dataframe
df = cd.read_csv('/kaggle/input/prediction-stock/IXIC.csv')
data = cd.DataFrame(data = df,columns=['Date','Open','High','Low','Close','Adj Close'])
data['target']=df.iloc[:,-1];
data

,Date,Open,High,Low,Close,Adj Close,target
0,1971-02-05,100.000000,100.000000,100.000000,100.000000,100.000000,0
1,1971-02-08,100.839996,100.839996,100.839996,100.839996,100.839996,0
2,1971-02-09,100.760002,100.760002,100.760002,100.760002,100.760002,0
3,1971-02-10,100.690002,100.690002,100.690002,100.690002,100.690002,0
4,1971-02-11,101.449997,101.449997,101.449997,101.449997,101.449997,0
...,...,...,...,...,...,...,...
12819,2021-12-02,15181.820313,15444.540039,15150.120117,15381.320313,15381.320313,5390100000
12820,2021-12-03,15428.709961,15470.360352,14931.059570,15085.469727,15085.469727,5859520000
12821,2021-12-06,15117.629883,15281.990234,14931.610352,15225.150391,15225.150391,5095960000
12822,2021-12-07,15510.910156,15720.089844,15507.660156,15686.919922,15686.919922,5091220000


In [4]:
data.isnull().sum()
type(data)

cudf.core.dataframe.DataFrame

In [5]:
# Date Feature is not needed so droping the date feature
df = data.drop(['Date'],axis = 1)

In [6]:
df

,Open,High,Low,Close,Adj Close,target
0,100.000000,100.000000,100.000000,100.000000,100.000000,0
1,100.839996,100.839996,100.839996,100.839996,100.839996,0
2,100.760002,100.760002,100.760002,100.760002,100.760002,0
3,100.690002,100.690002,100.690002,100.690002,100.690002,0
4,101.449997,101.449997,101.449997,101.449997,101.449997,0
...,...,...,...,...,...,...
12819,15181.820313,15444.540039,15150.120117,15381.320313,15381.320313,5390100000
12820,15428.709961,15470.360352,14931.059570,15085.469727,15085.469727,5859520000
12821,15117.629883,15281.990234,14931.610352,15225.150391,15225.150391,5095960000
12822,15510.910156,15720.089844,15507.660156,15686.919922,15686.919922,5091220000


In [7]:
# Splitting the data in X and Y 
# Findind the sizde of the dataset
df.shape

(12824, 6)

In [8]:
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]
Y = cp.asarray(Y)
Y=Y.reshape(-1,1)

In [9]:
# Standardization of Data
from cuml.preprocessing import StandardScaler
cuml_scaler = StandardScaler()
X_scaled = cuml_scaler.fit_transform(X)
X_scaled

,0,1,2,3,4
0,-0.720001,-0.720436,-0.719526,-0.719915,-0.719915
1,-0.719702,-0.720138,-0.719225,-0.719616,-0.719616
2,-0.719730,-0.720166,-0.719253,-0.719645,-0.719645
3,-0.719755,-0.720191,-0.719278,-0.719670,-0.719670
4,-0.719484,-0.719922,-0.719006,-0.719399,-0.719399
...,...,...,...,...,...
12819,4.655814,4.715540,4.683681,4.726573,4.726573
12820,4.743817,4.724688,4.605035,4.621127,4.621127
12821,4.632934,4.657955,4.605233,4.670911,4.670911
12822,4.773116,4.813157,4.812043,4.835493,4.835493


In [10]:
from cuml.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X_scaled,Y,test_size=0.3,random_state = 42)

In [11]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)
y_test = Y_test.astype('float64')

(8977, 5)
(3847, 5)
(8977, 1)
(3847, 1)


In [12]:
from cuml.linear_model import LinearRegression

In [13]:
#Fitting the model and findind the evalution metrics
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'eig')
reg = lr.fit(X_train,Y_train)
print("Coefficients:")
print(reg.coef_)
print("Intercept:")
print(reg.intercept_)
preds = lr.predict(X_test)
print("Predictions:")
print(preds)
print("MSE:")
print(ml.metrics.regression.mean_squared_error(y_test,preds))
print("R2 Score:")
print(ml.metrics.regression.r2_score(y_test,preds))
print("MAE:")
print(ml.metrics.regression.mean_absolute_error(y_test,preds))

Coefficients:
0   -6.574395e+08
1    1.873679e+10
2   -1.698525e+10
3   -5.431250e+07
4   -5.431250e+07
dtype: float64
Intercept:
1030420818.8010963
Predictions:
0       3.111108e+08
1       3.080969e+08
2       9.523869e+08
3       1.353283e+09
4       1.513327e+09
            ...     
3842    7.595691e+08
3843    5.320173e+09
3844    3.262338e+08
3845    6.406925e+08
3846    1.089654e+09
Length: 3847, dtype: float64
MSE:
2.3680946719560904e+18
R2 Score:
0.7506677041070462
MAE:
1129595805.0508757
